In [51]:
import pandas as pd
recipe_df = pd.read_csv('../data/test_recipes.csv')
product_df = pd.read_csv('../data/BigBasket.csv')
df_correct = pd.read_csv('../data/TestMappedData.csv')

In [52]:
recipe_df.head()

,Row,Name,Prep Time,Cook Time,Total Time,Servings,Yield,Ingredients,Directions,url,Additional Time
0,0,Indian Chicken Curry (Murgh Kari),20 mins,40 mins,1 hrs,6,6 servings,"[{'quantity': '2', 'unit': 'pounds', 'name': '...",['Sprinkle the chicken breasts with 2 teaspoon...,https://www.allrecipes.com/recipe/212721/india...,NaN
1,1,Chicken Stir-Fry,20 mins,20 mins,40 mins,6,NaN,"[{'quantity': '4', 'unit': 'cups', 'name': 'wa...",['Bring water and rice to a boil in a saucepan...,https://www.allrecipes.com/recipe/223382/chick...,NaN
2,2,Jambalaya,20 mins,45 mins,1 hrs 5 mins,6,6 servings,"[{'quantity': '2', 'unit': 'tablespoons', 'nam...",['Heat 1 tablespoon of peanut oil in a large h...,https://www.allrecipes.com/recipe/31848/jambal...,NaN
3,3,Chicken Katsu,10 mins,10 mins,20 mins,4,4 servings,"[{'quantity': '4', 'unit': '', 'name': 'skinle...",['Season the chicken breasts on both sides wit...,https://www.allrecipes.com/recipe/72068/chicke...,NaN
4,4,Simple Baked Chicken Breasts,10 mins,25 mins,35 mins,4,NaN,"[{'quantity': '4', 'unit': '(5 ounce)', 'name'...",['Preheat the oven to 400 degrees F (200 degre...,https://www.allrecipes.com/recipe/240208/simpl...,NaN


In [53]:
product_df.head()

,ProductName,Brand,Price,DiscountPrice,Image_Url,Quantity,Category,SubCategory,Absolute_Url
0,Onion (Loose),Fresho,69.75,52.0,https://www.bigbasket.com/media/uploads/p/l/40...,2 kg,Fruits & Vegetables,"Potato, Onion & Tomato",https://www.bigbasket.com/pd/40075537/fresho-o...
1,Onion,Fresho,174.35,130.0,https://www.bigbasket.com/media/uploads/p/l/12...,5 kg,Fruits & Vegetables,"Potato, Onion & Tomato",https://www.bigbasket.com/pd/1201414/fresho-on...
2,Onion (Loose),Fresho,34.87,26.0,https://www.bigbasket.com/media/uploads/p/l/10...,1 kg,Fruits & Vegetables,"Potato, Onion & Tomato",https://www.bigbasket.com/pd/10000148/fresho-o...
3,Onion,Fresho,69.74,52.0,https://www.bigbasket.com/media/uploads/p/l/12...,2 kg,Fruits & Vegetables,"Potato, Onion & Tomato",https://www.bigbasket.com/pd/1201413/fresho-on...
4,Onion (Loose),Fresho,174.37,129.0,https://www.bigbasket.com/media/uploads/p/l/10...,5 kg,Fruits & Vegetables,"Potato, Onion & Tomato",https://www.bigbasket.com/pd/10000150/fresho-o...


In [54]:
import ast
data = recipe_df['Ingredients'][2]
parsed_data = ast.literal_eval(data)
ingredients = [item['name'] for item in parsed_data]




In [55]:
edible_categories = [
    'Foodgrains, Oil & Masala', 
    'Gourmet & World Food', 
    'Snacks & Branded Foods', 
    'Eggs, Meat & Fish', 
    'Bakery, Cakes & Dairy', 
    'Beverages', 
    'Fruits & Vegetables'
]
product_name = product_df[product_df['Category'].isin(edible_categories)]['ProductName']

In [56]:
import pandas as pd
from fuzzywuzzy import process, fuzz

def map_ingredient_to_product_fuzzy(ingredient_list, product_list):
    mapped_products = []
    for ingredient in ingredient_list:
        # Using process.extractOne to get the best match
        best_match = process.extractOne(ingredient, product_list, scorer=fuzz.token_sort_ratio)
        mapped_products.append((ingredient, best_match[0]))  # (ingredient, matched product)
    
    return pd.DataFrame({
        'Ingredient': [item[0] for item in mapped_products],
        'Product': [item[1] for item in mapped_products]
    })

In [57]:
product_list = list(product_name)

In [58]:
df_fuzzy = map_ingredient_to_product_fuzzy(ingredients,product_list)
df_fuzzy.head(5)

,Ingredient,Product
0,"skinless, boneless chicken breast halves - pou...",Chicken Curry Cut Without Skin 500 g + Chicken...
1,salt and pepper to taste,Masters Blend Tea- Rich Taste
2,all-purpose flour,Rice - Flour
3,"egg, beaten",Green Peas
4,panko bread crumbs,Caperberries in Brine


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def map_ingredients_to_products_cosine(ingredients, product_list):
    # Combine ingredients and product names into a single list
    combined_list = product_list + ingredients

    # Convert text to TF-IDF vectors
    vectorizer = TfidfVectorizer().fit_transform(combined_list)
    vectors = vectorizer.toarray()

    # Calculate cosine similarity between ingredients and products
    similarity_matrix = cosine_similarity(vectors[len(product_list):], vectors[:len(product_list)])

    # Find best matches
    best_matches = similarity_matrix.argmax(axis=1)

    # Create a DataFrame with the mapping
    mapping = pd.DataFrame({
        'Ingredient': ingredients,
        'Product': [product_list[i] for i in best_matches]
    })

    return mapping

In [60]:
df_cosine = map_ingredients_to_products_cosine(ingredients,product_list)
df_cosine

,Ingredient,Product
0,"skinless, boneless chicken breast halves - pou...","Chicken Breast - Boneless, Antibiotic Residue-..."
1,salt and pepper to taste,Potato Finger Salt
2,all-purpose flour,Multi-Purpose Seasoning
3,"egg, beaten","Avalakki Medium - Flattened & Beaten Rice , M..."
4,panko bread crumbs,Bread Sticks - Plain
5,"oil for frying, or as needed",Olive Oil For Indian Cooking - Frying & Baking


In [61]:
df_fuzzy_filtered = df_fuzzy[df_fuzzy['Ingredient'].isin(df_correct['Ingredient'])]
df_merged = df_fuzzy_filtered.merge(df_correct, on='Ingredient', how='left', suffixes=('_fuzzy', '_correct'))
df_merged['evaluation'] = df_merged.apply(lambda row: 'Correct' if row['Product_fuzzy'] == row['Product_correct'] 
                                          else 'Mismatch', axis=1)
df_unmatched = df_fuzzy[~df_fuzzy['Ingredient'].isin(df_correct['Ingredient'])]
df_unmatched['product_correct'] = 'Not found'
df_unmatched['evaluation'] = 'Unknown'
df_final = pd.concat([df_merged, df_unmatched], ignore_index=True)
correct_matches = df_final[df_final['evaluation'] == 'Correct'].shape[0]
total_ingredients = df_final.shape[0]
accuracy = (correct_matches / total_ingredients) * 100

# Output the final dataframe and accuracy
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 0.00%


In [62]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_match(df_algorithm, df_correct):
    """
    Evaluates the performance of a dataframe generated from an algorithm by comparing it to the correct dataframe.
    
    Parameters:
    - df_algorithm: DataFrame containing ingredient-to-product mappings produced by a matching algorithm.
    - df_correct: DataFrame containing the correct ingredient-to-product mappings.
    
    Returns:
    A dictionary containing accuracy, precision, recall, F1 score, and other evaluation metrics.
    """

    # Step 1: Filter df_algorithm for ingredients that are present in the correct dataframe
    df_algorithm_filtered = df_algorithm[df_algorithm['Ingredient'].isin(df_correct['Ingredient'])]

    # Step 2: Merge the filtered df_algorithm with the correct dataframe
    df_merged = df_algorithm_filtered.merge(df_correct, on='Ingredient', how='left', suffixes=('_algo', '_correct'))

    # Step 3: Create a new column to evaluate match status
    df_merged['evaluation'] = df_merged.apply(lambda row: 'Correct' if row['Product_algo'] == row['Product_correct'] 
                                              else 'Mismatch', axis=1)

    # Step 4: Add unmatched ingredients (those not found in the correct dataframe)
    df_unmatched = df_algorithm[~df_algorithm['Ingredient'].isin(df_correct['Ingredient'])]
    df_unmatched['Product_correct'] = 'Not found'
    df_unmatched['evaluation'] = 'Unknown'

    # Step 5: Concatenate the matched and unmatched dataframes
    df_final = pd.concat([df_merged, df_unmatched], ignore_index=True)

    # Step 6: Calculate accuracy based on 'Correct' and 'Mismatch'
    correct_matches = df_final[df_final['evaluation'] == 'Correct'].shape[0]
    total_ingredients = df_final.shape[0]
    accuracy = (correct_matches / total_ingredients) * 100

    # Step 7: Extract true and predicted labels (only for the matched ingredients)
    y_true = df_final[df_final['evaluation'] != 'Unknown']['Product_correct']
    y_pred = df_final[df_final['evaluation'] != 'Unknown']['Product_algo']

    # Step 8: Calculate other metrics (precision, recall, F1)
    accuracy_eval = accuracy_score(y_true, y_pred)
    precision_eval = precision_score(y_true, y_pred, average='macro', zero_division=1)
    recall_eval = recall_score(y_true, y_pred, average='macro', zero_division=1)
    f1_eval = f1_score(y_true, y_pred, average='macro', zero_division=1)

    # Return a dictionary with all the metrics
    return {
        'accuracy': accuracy,
        'precision': precision_eval,
        'recall': recall_eval,
        'f1_score': f1_eval,
        'total_ingredients': total_ingredients,
        'correct_matches': correct_matches,
        'final_dataframe': df_final
    }

In [63]:
# Example usage with sample dataframes

metrics = evaluate_match(df_fuzzy, df_correct)

# Output the metrics
print(f"Accuracy: {metrics['accuracy']:.2f}")
print(f"Precision: {metrics['precision']:.2f}")
print(f"Recall: {metrics['recall']:.2f}")
print(f"F1 Score: {metrics['f1_score']:.2f}")
print(f"Correct Matches: {metrics['correct_matches']}")
print(f"Total Ingredients Evaluated: {metrics['total_ingredients']}")


Accuracy: 0.00
Precision: nan
Recall: nan
F1 Score: nan
Correct Matches: 0
Total Ingredients Evaluated: 6


/home/mirsahib/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/mirsahib/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [64]:

metrics = evaluate_match(df_cosine, df_correct)

# Output the metrics
print(f"Accuracy: {metrics['accuracy']:.2f}")
print(f"Precision: {metrics['precision']:.2f}")
print(f"Recall: {metrics['recall']:.2f}")
print(f"F1 Score: {metrics['f1_score']:.2f}")
print(f"Correct Matches: {metrics['correct_matches']}")
print(f"Total Ingredients Evaluated: {metrics['total_ingredients']}")

Accuracy: 0.00
Precision: nan
Recall: nan
F1 Score: nan
Correct Matches: 0
Total Ingredients Evaluated: 6


/home/mirsahib/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/mirsahib/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [65]:

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def map_ingredients_to_products_sementic(ingredients_df, products_df, 
                     ingredient_col='ingredient_name', 
                     product_col='product_name',
                     threshold=0.6,
                     model_name='all-MiniLM-L6-v2'):
    """
    Find the best matching product for each ingredient using semantic similarity.
    
    Parameters:
    -----------
    ingredients_df : pandas.DataFrame
        DataFrame containing ingredients
    products_df : pandas.DataFrame
        DataFrame containing products
    ingredient_col : str
        Name of the column containing ingredient names
    product_col : str
        Name of the column containing product names
    threshold : float
        Minimum similarity score to consider a match
    model_name : str
        Name of the sentence transformer model to use
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with best matched product for each ingredient
    """
    # Load the sentence transformer model
    model = SentenceTransformer(model_name)
    
    # Get unique ingredients and products
    ingredients = ingredients_df[ingredient_col].unique()
    products = products_df[product_col].unique()
    
    # Generate embeddings
    ingredient_embeddings = model.encode(ingredients, show_progress_bar=False)
    product_embeddings = model.encode(products, show_progress_bar=False)
    
    # Calculate similarity matrix
    similarity_matrix = cosine_similarity(ingredient_embeddings, product_embeddings)
    
    # Find best matches
    best_matches = []
    for i, ingredient in enumerate(ingredients):
        # Get similarities for this ingredient
        similarities = similarity_matrix[i]
        
        # Get best match index
        best_match_idx = np.argmax(similarities)
        best_score = similarities[best_match_idx]
        
        if best_score >= threshold:
            best_matches.append({
                'Ingredient': ingredient,
                'Product': products[best_match_idx],
                'Similarity Score': round(best_score, 3)

            })
        else:
            best_matches.append({
                'Ingredient': ingredient,
                'Product': 'No match found',
                'Similarity Score': round(best_score, 3)

            })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(best_matches)
    
    return results_df



In [66]:
# Example usage with your sample data
ingredients_data = {
    'ingredient_name': ingredients
}

products_data = {
    'product_name': product_list
}

ingredients_df = pd.DataFrame(ingredients_data)
products_df = pd.DataFrame(products_data)

# Get best matches
best_matches = map_ingredients_to_products_sementic(ingredients_df, products_df, threshold=0.6)
best_matches

,Ingredient,Product,Similarity Score
0,"skinless, boneless chicken breast halves - pou...","Chicken - Mixed Boneless, Without Skin",0.633
1,salt and pepper to taste,No match found,0.541
2,all-purpose flour,Whole Wheat Flour,0.745
3,"egg, beaten",No match found,0.558
4,panko bread crumbs,Bread Sticks - Plain,0.609
5,"oil for frying, or as needed","Extra Light Olive Oil - Frying, Sauteing & Gri...",0.627


In [67]:

metrics = evaluate_match(best_matches, df_correct)

# Output the metrics
print(f"Accuracy: {metrics['accuracy']:.2f}")
print(f"Precision: {metrics['precision']:.2f}")
print(f"Recall: {metrics['recall']:.2f}")
print(f"F1 Score: {metrics['f1_score']:.2f}")
print(f"Correct Matches: {metrics['correct_matches']}")
print(f"Total Ingredients Evaluated: {metrics['total_ingredients']}")

Accuracy: 0.00
Precision: nan
Recall: nan
F1 Score: nan
Correct Matches: 0
Total Ingredients Evaluated: 6


/home/mirsahib/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/mirsahib/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
